In [37]:
import requests
from bs4 import BeautifulSoup
import random
import time

In [34]:
HEADERS = [{
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
        },
    
        {
        "Sec-Ch-Ua": '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
        "Sec-Ch-Ua-Mobile": "?1",
        "Sec-Ch-Ua-Platform": '"Android"',
        "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Mobile Safari/537.36"
        }
]


delay_seconds = 2
start_index = 0 ### to start from 0th index
pages_per_city = 1
review_per_restaurant_number = 20  ### better if given in multiple of 10
review_per_restaurant_percent = 0.5
review_per_restaurant_see_number = True
base_url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York&start="

In [29]:
def get_random_header():
    ran_index = random.randint(0, len(HEADERS) - 1)
    return HEADERS[ran_index]

In [35]:
### returns True if we need to stop scraping else return False:
def check_done(total_review_pages, view_pages):
    if( (view_pages +1) > total_review_pages):
        return True
    
    if(review_per_restaurant_see_number):
        if( ((view_pages+1) * 10) > review_per_restaurant_number):
            return True
        else:
            return False
    
    else:
        if( (view_pages+1)> review_per_restaurant_percent* total_review_pages):
            return True
        else:
            return False
        
        

In [63]:
def start_scraping():
    main_data_list = []
    for page in range(pages_per_city):
        current_url = base_url + str(page)
        headers = get_random_header()
        res = requests.get(current_url)
        
        ###parsing and analysisis:
        htmlData = res.text
        parsedData = BeautifulSoup(htmlData, 'html.parser')
        restaurant_links = parsedData.css.select("main > div >ul > li > div > div > div > div:nth-of-type(2) > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(1)> div a")
        restaurant_names = [each_link.text for each_link in restaurant_links]
        print("Restaurant Names = " ,restaurant_names)
        
        ####for every restaurant:
        for each_restaurant in restaurant_links:
            print("##############################################")
            print("Restaurant Name = ", each_restaurant.text)
            base_url_restaurant = "https://www.yelp.com"+ each_restaurant["href"] + "&start="
            
            ### for every pages of reviews:
            view_pages = 0
            total_review_pages = 1 #just for initial case:
            while(not check_done(total_review_pages, view_pages)):
                temp_row_list = []
                
                current_url_restaurant = base_url_restaurant + "1"
                res_each_page = requests.get(current_url_restaurant)
                htmlData2 = res_each_page.text
                parsedData2 =BeautifulSoup(htmlData2, 'html.parser')

                ### finding the total number of review pages for each restaurant:::
                if(view_pages == 0):
                    total_review_pages_string = parsedData2.select_one(".pagination__09f24__VRjN4 > div:nth-of-type(2) span").text
                    print("total_review_pages_string= " ,total_review_pages_string)
                    total_review_pages = int(total_review_pages_string.split("of")[1])

                #extraing the final reviews:
                parsed_reviews = parsedData2.select("main >div >div> section> div>ul>li")
                
                for each_parsed_reviews in parsed_reviews:
                    username = each_parsed_reviews.select_one("div >span > a ").text
                    user_address = each_parsed_reviews.select("div > .css-qgunke")[0].text
                    rating = each_parsed_reviews.select("span >  .five-stars__09f24__mBKym")[0]["aria-label"]
                    review_date = each_parsed_reviews.select("div > .css-chan6m")[0].text
                    review_text = each_parsed_reviews.select("div > .css-qgunke")[1].text

                    temp_row_list.append(username)
                    temp_row_list.append(user_address)
                    temp_row_list.append(rating)
                    temp_row_list.append(review_date)
                    temp_row_list.append(review_text)
                    main_data_list.append(temp_row_list)
                
                view_pages += 1
                print("Reviw page done: ", view_pages)
                
                #delaying :
                time.sleep(delay_seconds)
            
            ##
            print("Restaurant done: ")
            print("##############################################")
        
        ##
        print("Page done: " ,page)
        return main_data_list
    

In [64]:
main_data_list = start_scraping()

Restaurant Names =  []
Page done:  0


In [43]:
main_data_list

[]